# Lab Exercise 2: Unconstrained Least Squares Unmixing

## Objectives
- Implement the analytical solution for unconstrained least squares
- Apply it to hyperspectral data
- Evaluate reconstruction quality
- Understand limitations of unconstrained approach

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

from data_loader import HyperspectralDataLoader
from visualization import HSIVisualizer
from optimization import HyperspectralUnmixer
from metrics import UnmixingEvaluator

%matplotlib inline

## Task 1: Load Data and Endmembers

Load the previously processed data and endmembers.

In [ ]:
# Load data
loader = HyperspectralDataLoader("../data/")
hsi_data, ground_truth = loader.load_pavia()
Y, _ = loader.vectorize_data()

# Load endmembers from previous exercise
try:
    S = np.load('../data/extracted_endmembers.npy')
    endmember_names = np.load('../data/endmember_names.npy', allow_pickle=True)
    print(f"Loaded endmembers: {S.shape}")
    print(f"Endmember names: {list(endmember_names)}")
except FileNotFoundError:
    print("Endmembers not found. Re-extracting...")
    S, endmember_names = loader.extract_class_spectra([1, 2, 3])

print(f"Data matrix Y shape: {Y.shape}")
print(f"Endmember matrix S shape: {S.shape}")

## Task 2: Implement Unconstrained Least Squares

**Mathematical Background:**

The unconstrained least squares problem is:
$$\min_{\mathbf{A}} \|\mathbf{S}\mathbf{A} - \mathbf{Y}\|_F^2$$


In [ ]:
def unconstrained_least_squares_manual(S, Y):
    """
    Manual implementation of unconstrained least squares.
    
    TODO: Implement the analytical solution
    """
    # Step 1: Compute S^T S
    StS = ## YOUR CODE HERE
    
    # Step 2: Compute S^T Y  
    StY = ## YOUR CODE HERE
    
    # Step 3: Solve the LS problem
    try:
        A =  # YOUR CODE HERE (use np.linalg.solve)
    except np.linalg.LinAlgError:
        print("Warning: Using pseudo-inverse due to singular matrix")
        A = # YOUR CODE HERE (use np.linalg.pinv)
    
    return A

# Test your implementation
A_manual = unconstrained_least_squares_manual(S, Y)
print(f"Abundance matrix shape: {A_manual.shape}")

## Task 3: Compare with Library Implementation

Compare your manual implementation with the provided class method.

In [ ]:
# Use the provided implementation
unmixer = HyperspectralUnmixer()
A_library = unmixer.unconstrained_least_squares(S, Y)

# Compare results
difference = np.max(np.abs(A_manual - A_library))
print(f"Maximum difference between implementations: {difference:.2e}")

if difference < 1e-10:
    print("✓ Implementations match!")
else:
    print("✗ Implementations differ - check your code")

## Task 5: Visualize Results

Create visualizations to understand the unmixing results.

In [ ]:
# TODO: Plot abundance maps
visualizer = HSIVisualizer()
height, width = hsi_data.shape[:2]

# YOUR CODE HERE

## Task 6: Evaluate Reconstruction Quality

Compute reconstruction metrics to assess unmixing quality.

In [ ]:
# TODO: Evaluate reconstruction
## YOUR CODE HERE

results = ## YOUR CODE HERE

print("Reconstruction Quality Metrics:")
print(f"  Mean SAM (degrees): {results['mean_sam_degrees']:.4f}")
print(f"  RMSE: {results['rmse']:.6f}")
if 'ssim_mean' in results:
    print(f"  Mean SSIM: {results['ssim_mean']:.4f}")
print(f"  SNR (dB): {results['snr_db']:.2f}")

## Task 7: Visualize Reconstruction

Compare original and reconstructed RGB composites.

In [ ]:
# TODO: Reconstruct hyperspectral data
## YOUR CODE HERE

# TODO: Plot comparison
# YOUR CODE HERE


In [ ]:
# Save results for comparison in next notebook
np.save('../data/unconstrained_abundances.npy', A_library)
print("Results saved for comparison!")